In [1]:
import torch
print (torch.cuda.is_available())

True


In [2]:
from huggingface_hub import hf_hub_download
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to C:\Users\researcher\.huggingface\token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [3]:
from datasets import load_dataset

# Load dataset

In [4]:
data_files = { "raw_data":"200_samplecontent.csv"}
df_dataset=load_dataset("HFFErica/steamreviews", data_files=data_files)
df_dataset

Using custom data configuration HFFErica--steamreviews-438ffa7a173e0b5d


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

D:\Software\Anaconda3\envs\pytorch\lib\site-packages\datasets\download\streaming_download_manager.py:697: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to C:/Users/researcher/.cache/huggingface/datasets/HFFErica___csv/HFFErica--steamreviews-438ffa7a173e0b5d/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    raw_data: Dataset({
        features: ['No.1', 'No.', '_id', 'time', 'content', 'Automated Readability Index', 'The Flesch Reading Ease formula', 'The Flesch-Kincaid Grade Level', 'The SMOG Index', 'The Coleman-Liau Index', 'Dale-Chall Readability Score', 'Linsear Write Formula', 'The Fog Scale (Gunning FOG Formula)', 'difficult_words', 'Readability Consensus based upon all the above tests', 'item_count', 'recommend', 'found_helpful', 'found_funny', 'reply_count', 'tags.Heartwarming', 'hrs_on_second', 'hrs_at_reviwer_time', 'reviwer_num'],
        num_rows: 200
    })
})

In [7]:
content = df_dataset['raw_data']['content']
content [1]

"Game ran like crap at first but a patch less then 24 hours after release fixed 95% of my problems. No other developer can deliver patches that quick these days.As far as the game goes, after almost 20 hours I'm still thoroughly enjoying it. It is nice to have a game that doesn't push you towards the end and allows you to play as your own pace while creating your own goals."

# DPP

In [8]:
from bs4 import BeautifulSoup
import spacy
import nltk
import re
import string
import pandas as pd
import numpy as np
# from stop_word_list import *
#from clean_text import *
import gensim
from gensim import corpora
# import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import json
%matplotlib inline

In [9]:
content [1]= re.sub(r"http\S+", "", content [1])

In [10]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"was\'t", "was not", phrase)
    phrase = re.sub(r"gonna", "going to", phrase)
    phrase = re.sub(r"dont", "do not", phrase)
    phrase = re.sub(r"doesnt ", "does not", phrase)
    phrase = re.sub(r" didnt ", " did not ", phrase)
    phrase = re.sub(r" havent ", " have not ", phrase)
    phrase = re.sub(r" cant ", " can not ", phrase)
    phrase = re.sub(r" couldnt ", " could not ", phrase)
    phrase = re.sub(r" wouldnt ", " would not ", phrase)
    phrase = re.sub(r" thats ", " that is ", phrase)
    phrase = re.sub(r" isnt ", " is not ", phrase)
    phrase = re.sub(r" arent ", " are not ", phrase)
    phrase = re.sub(r" ive ", " i have ", phrase)
    phrase = re.sub(r" iam ", " i am ", phrase)
    phrase = re.sub(r" dev ", " developer ", phrase)
    phrase = re.sub(r" devs ", " developers ", phrase)
    phrase = re.sub(r" thx ", " thanks ", phrase)
    phrase = re.sub(r" btw ", " by the way ", phrase)
    phrase = re.sub(r" pc ", " personal computer ", phrase)
    phrase = re.sub(r" ur ", " your are ", phrase)
    phrase = re.sub(r" hrs ", " hours ", phrase)
    phrase = re.sub(r" hr ", " hour ", phrase)
    phrase = re.sub(r" E3 ", " electronic entertainment expo ", phrase)
    phrase = re.sub(r" e3 ", " electronic entertainment expo ", phrase)
    phrase = re.sub(r" nms ", " no mans sky ", phrase)
    phrase = re.sub(r" pve ", " player versus environment ", phrase)
    phrase = re.sub(r" pvp ", " player versus player ", phrase)
    phrase = re.sub(r" mmo ", " massively multiplayer online game ", phrase)
    phrase = re.sub(r" tv ", " television ", phrase)
    phrase = re.sub(r" rpg ", " role-playing video game ", phrase)
    phrase = re.sub(r" wp ", " well played ", phrase)
    phrase = re.sub(r" gg ", " good game ", phrase)
    phrase = re.sub(r" dlc ", " downloadable content ", phrase)
    phrase = re.sub(r" npc ", " non-player character ", phrase)
    phrase = re.sub(r" hg ", " hello games ", phrase)
    phrase = re.sub(r" hrs ", " hours ", phrase)
    phrase = re.sub(r" btw ", " by the way ", phrase)
    phrase = re.sub(r" mph ", " miles per hour ", phrase) 
    phrase = re.sub(r" fov ", " field of view ", phrase) 
    phrase = re.sub(r" rly ", " really ", phrase) 
    phrase = re.sub(r" em ", " them ", phrase) 
    phrase = re.sub(r" imho ", " in my humble opinion ", phrase) 
    phrase = re.sub(r" tl;dr ", " too long; did not read ", phrase) 
    phrase = re.sub(r" tldr ", " too long; did not read ", phrase) 
    phrase = re.sub(r" imao ", " laughing my ass off ", phrase) 
    phrase = re.sub(r" bs ", " bullshit ", phrase) 
    phrase = re.sub(r" tbh ", " to be honest ", phrase)
    phrase = re.sub(r" cuz ", " because ", phrase) 
    phrase = re.sub(r" E:D ", " elite dangerous ", phrase)
    phrase = re.sub(r" e:d ", " elite dangerous ", phrase) 
    phrase = re.sub(r" ad ", " advantages ", phrase) 
    phrase = re.sub(r" kok ", " king of kings ", phrase) 
    phrase = re.sub(r" wmr ", " with much respect ", phrase) 
    phrase = re.sub(r" tfw ", " that feel when ", phrase) 
    phrase = re.sub(r" f2p ", " free to play ", phrase) 
    phrase = re.sub(r" lmb ", " left mouse button ", phrase) 
    phrase = re.sub(r" eve ", " everyone versus everyone ", phrase) 
    phrase = re.sub(r" gui ", " game user interface ", phrase) 
    phrase = re.sub(r" c-mon ", " come on ", phrase) 
    phrase = re.sub(r" c'mon ", " come on ", phrase) 
    
    
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [11]:
def to_lower(sent) :
    return sent.lower()

In [12]:
stopwords= set([])
# stopwords= set(["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"])

In [13]:
def encode_emojis(s):
    s = re.sub(r'♥+', 'profanity', s)
    s = s.replace(':)', 'smiley')
    s = s.replace(':D ', 'laughing')
    s = s.replace(' xD ', 'laughing')
    s = s.replace(' x‑D ', 'laughing')
    s = s.replace(' :"D ', 'tears of happiness')
    s = s.replace(' :O ', 'surprise')
    s = s.replace(':)', 'smiley')
    s = s.replace(':(', 'frowney')
    s = s.replace('<3', 'heart')
    return s

In [14]:
# Combining all the above. 
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(content):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = to_lower (sentance)
    # sentance = lemmatize (sentance)
    sentance = decontracted(sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = encode_emojis(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    # sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

  0%|                                                                                          | 0/200 [00:00<?, ?it/s]D:\Software\Anaconda3\envs\pytorch\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 1775.53it/s]


In [15]:
content[1]

"Game ran like crap at first but a patch less then 24 hours after release fixed 95% of my problems. No other developer can deliver patches that quick these days.As far as the game goes, after almost 20 hours I'm still thoroughly enjoying it. It is nice to have a game that doesn't push you towards the end and allows you to play as your own pace while creating your own goals."

In [16]:
preprocessed_reviews[1]

'game ran like crap at first but a patch less then hours after release fixed of my problems. no other developer can deliver patches that quick these days.as far as the game goes, after almost hours i am still thoroughly enjoying it. it is nice to have a game that does not push you towards the end and allows you to play as your own pace while creating your own goals.'

In [19]:
df_dataset['preprocessed_reviews'] =preprocessed_reviews

In [22]:
import pandas as pd
name=['preprocessed']
preprocessed_data=pd.DataFrame(columns=name,data=preprocessed_reviews)
print(preprocessed_data)

                                          preprocessed
0                              i am your xiao tiantian
1    game ran like crap at first but a patch less t...
2    the game has come a long way since release. i ...
3    it is the most amazing conception created by m...
4                              nope, not in this state
..                                                 ...
195  * post-next patch review *i am calling the nex...
196  an early access indie game with little to no g...
197  "no mans lie" this is how i knew this game for...
198  no man is sky had very many features missing f...
199  love this game if you like exploring give it a...

[200 rows x 1 columns]


# Model

## Two labels

In [23]:
from transformers import pipeline

ppd= preprocessed_data['preprocessed']
classifier = pipeline('zero-shot-classification')
labels =['video games','refund']
# 'contents','space','patch','running isses', 'developers', 'enjoyment', 'bug','promotion',
results = classifier(ppd.to_list(), labels, multi_class=True)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


In [24]:
results[1]

{'sequence': 'game ran like crap at first but a patch less then hours after release fixed of my problems. no other developer can deliver patches that quick these days.as far as the game goes, after almost hours i am still thoroughly enjoying it. it is nice to have a game that does not push you towards the end and allows you to play as your own pace while creating your own goals.',
 'labels': ['video games', 'refund'],
 'scores': [0.9867268800735474, 0.10791972279548645]}

## Add labels

In [25]:
ppd= preprocessed_data['preprocessed']
# ppd[0]
classifier = pipeline('zero-shot-classification')
labels =['video games','space','update','operation','explore', 'developers', 'enjoyment', 'bug','promotion','refund']
results = classifier(ppd.to_list(), labels, multi_class=True)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


In [26]:
results[:2]

[{'sequence': 'i am your xiao tiantian',
  'labels': ['promotion',
   'update',
   'operation',
   'enjoyment',
   'refund',
   'bug',
   'explore',
   'video games',
   'developers',
   'space'],
  'scores': [0.44755715131759644,
   0.4319249391555786,
   0.41336172819137573,
   0.31854918599128723,
   0.22503018379211426,
   0.06361367553472519,
   0.033668700605630875,
   0.01616782508790493,
   0.011091919615864754,
   0.010751301422715187]},
 {'sequence': 'game ran like crap at first but a patch less then hours after release fixed of my problems. no other developer can deliver patches that quick these days.as far as the game goes, after almost hours i am still thoroughly enjoying it. it is nice to have a game that does not push you towards the end and allows you to play as your own pace while creating your own goals.',
  'labels': ['video games',
   'enjoyment',
   'developers',
   'explore',
   'space',
   'update',
   'operation',
   'bug',
   'promotion',
   'refund'],
  'score

In [27]:
df_dataset['label_results'] = results

In [28]:
df_dataset['label_results'][1]

{'sequence': 'game ran like crap at first but a patch less then hours after release fixed of my problems. no other developer can deliver patches that quick these days.as far as the game goes, after almost hours i am still thoroughly enjoying it. it is nice to have a game that does not push you towards the end and allows you to play as your own pace while creating your own goals.',
 'labels': ['video games',
  'enjoyment',
  'developers',
  'explore',
  'space',
  'update',
  'operation',
  'bug',
  'promotion',
  'refund'],
 'scores': [0.9867268800735474,
  0.9743295907974243,
  0.7993965744972229,
  0.5485067963600159,
  0.4035073220729828,
  0.3606005907058716,
  0.34618768095970154,
  0.2101818323135376,
  0.11741180717945099,
  0.10791972279548645]}

In [29]:
label_results=df_dataset['label_results']

In [30]:
name=['labels','scores','sequence']
label_results=pd.DataFrame(columns=name,data=label_results)
print(label_results)
label_results.to_csv('./results/label_results.csv',encoding='utf-8', index=False)

                                                labels  \
0    [promotion, update, operation, enjoyment, refu...   
1    [video games, enjoyment, developers, explore, ...   
2    [video games, update, enjoyment, explore, deve...   
3    [video games, enjoyment, developers, bug, upda...   
4    [bug, developers, refund, operation, update, p...   
..                                                 ...   
195  [video games, refund, enjoyment, operation, bu...   
196  [video games, developers, bug, explore, operat...   
197  [video games, developers, operation, promotion...   
198  [video games, space, developers, explore, oper...   
199  [explore, enjoyment, video games, space, devel...   

                                                scores  \
0    [0.44755715131759644, 0.4319249391555786, 0.41...   
1    [0.9867268800735474, 0.9743295907974243, 0.799...   
2    [0.985668420791626, 0.846666157245636, 0.67381...   
3    [0.8799867630004883, 0.7243720293045044, 0.596...   
4    [0.47797